## **PromptTemplate과 ChatPromptTemplate**

**[PromptTemplate]**

In [ ]:
from langchain.prompts import PromptTemplate

prompt= (
    PromptTemplate.from_template(
        """
        너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 {개수}추천하고,
        그 요리의 레시피를 제시해줘. 내가 가진 재료는 아래와 같아.
        <재료>
        {재료}
        """
        )
    )

prompt

In [ ]:
prompt.format(개수= 3, 재료="사과, 양파, 계란")

**[ChatPromptTemplate]**

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
prompt = SystemMessage(content=
        """
        너는 항상 밝은 말투로 대화하는 챗봇이야. 답변의 끝에 이모티콘을 붙여줘.
        """
        )
new_prompt = (
    prompt
    + HumanMessage(content=
                          """
                          오늘은 날씨가 어때?
                          """)
    + AIMessage(content=
                         """
                         오늘은 날씨가 아주 좋아요!
                         """)
    + """{input}"""
)
new_prompt.format_messages(input = "오늘 너의 기분은 어때?")

**[간단한 LLM Chain 구성해보기]**

In [ ]:
import os
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

model = ChatOpenAI(model = 'gpt-4o-mini')
chain = LLMChain(llm=model, prompt=new_prompt)
chain.invoke("오늘 너의 기분은 어때?")

## **Few-shot 예제를 통한 프롬프트 템플릿**

**[3행시 예제 프롬프트 템플릿]**

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
examples = [
    {
        "question": "아이유로 삼행시 만들어줘",
        "answer":
                """
                아: 아이유는
                이: 이런 강의를 들을 이
                유: 유가 없다.
                """
    }
]

example_prompt = PromptTemplate(
                                input_variables=["question", "answer"],
                                template="Question: {question}\n{answer}"
                                )

print(example_prompt.format(**examples[0]))

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(prompt.format(input="호날두로 삼행시 만들어줘"))

**[Few-shot 미적용 결과]**

In [ ]:
model = ChatOpenAI(model_name = "gpt-4o-mini", temperature = 1)
result = model.invoke("호날두로 삼행시 만들어줘")
print(result.content)

**[Few-shot 적용 결과]**

In [ ]:
result = model.invoke(prompt.format(input="호날두로 삼행시 만들어줘"))
print(result.content)

## **Partial Prompt Template**

**[Partial Prompt 예시]**

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("나이: {age} \n직업: {job}")
partial_prompt = prompt.partial(age="20")
print(partial_prompt.format(job="개발자"))


**[현재 날짜 Partial Prompt 만들어보기]**

In [ ]:
from datetime import datetime


def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")

In [ ]:
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective", "date"],
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))